# **Launch Sites Locations Analysis with Folium**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

Let's first import required Python packages for this lab:

In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster

# Import folium MousePosition plugin
from folium.plugins import MousePosition

# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:

[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)

In [3]:
## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df=pd.read_csv(URL)
spacex_df.head(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
5,7,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170.0,GTO,SES,No attempt,0,28.562302,-80.577356
6,8,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325.0,GTO,Thaicom,No attempt,0,28.562302,-80.577356
7,9,2014-04-18,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296.0,LEO (ISS),NASA (CRS),Controlled (ocean),0,28.562302,-80.577356
8,10,2014-07-14,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316.0,LEO,Orbcomm,Controlled (ocean),0,28.562302,-80.577356
9,11,2014-08-05,8:00:00,F9 v1.1,CCAFS LC-40,AsiaSat 8,4535.0,GTO,AsiaSat,No attempt,0,28.562302,-80.577356


In [5]:
spacex_df['Date'] = pd.to_datetime(spacex_df['Date'])

In [6]:
spacex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Flight Number      56 non-null     int64         
 1   Date               56 non-null     datetime64[ns]
 2   Time (UTC)         56 non-null     object        
 3   Booster Version    56 non-null     object        
 4   Launch Site        56 non-null     object        
 5   Payload            56 non-null     object        
 6   Payload Mass (kg)  56 non-null     float64       
 7   Orbit              56 non-null     object        
 8   Customer           56 non-null     object        
 9   Landing Outcome    56 non-null     object        
 10  class              56 non-null     int64         
 11  Lat                56 non-null     float64       
 12  Long               56 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
memory usage:

In [7]:
spacex_df['Year'] = spacex_df['Date'].dt.year
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,Year
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356,2010
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356,2010
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356,2012
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356,2012
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356,2013


In [8]:
spacex_df.groupby('Launch Site')['Year'].mean()

Launch Site
CCAFS LC-40     2014.269231
CCAFS SLC-40    2017.857143
KSC LC-39A      2017.076923
VAFB SLC-4E     2016.800000
Name: Year, dtype: float64

In [9]:
spacex_df.groupby('Launch Site')['Flight Number'].count()

Launch Site
CCAFS LC-40     26
CCAFS SLC-40     7
KSC LC-39A      13
VAFB SLC-4E     10
Name: Flight Number, dtype: int64

Now, you can take a look at what are the coordinates for each site.

In [10]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`

spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_site_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_site_df = launch_site_df[['Launch Site', 'Lat', 'Long']]
launch_site_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [11]:
# Start location is NASA Johnson Space Center

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name

circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name

marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

An example of folium.Circle:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

An example of folium.Marker:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [13]:
launch_site_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


In [14]:
# Initial the map

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for site, lat, lng in zip(launch_site_df['Launch Site'], launch_site_df['Lat'], launch_site_df['Long']):
    circle2 = folium.Circle([lat, lng], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(site))
    marker2 = folium.map.Marker([lat, lng], 
                                icon=DivIcon(
                                    icon_size=(30, 30), 
                                    icon_anchor=(0,0),
                                    html='<div style="font-size: 15; color:#d35400;"><b>%s</b></div>' % site,
                                    )
                               )
    site_map.add_child(circle2) # 얘도 for 문 안에 있어야 site_map 개체에 저 위에 4개의 위치가 추가가 되게 된다. for 문 밖에 있으면 실컷 4 개의 위치는 돌았어도 중 마지막 혹은 처음만 site_map 개체에 추가 되는 것임...
    site_map.add_child(marker2)

site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

In [15]:
# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [17]:
success_rate_eachsite = spacex_df.groupby('Launch Site')['class'].mean()

success_rate_eachsite

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [18]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value

In [19]:
spacex_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0
5,CCAFS LC-40,28.562302,-80.577356,0
6,CCAFS LC-40,28.562302,-80.577356,0
7,CCAFS LC-40,28.562302,-80.577356,0
8,CCAFS LC-40,28.562302,-80.577356,0
9,CCAFS LC-40,28.562302,-80.577356,0


In [20]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def color(x):
    if x == 1:
        return 'green'
    else:
        return 'red'

In [21]:
spacex_df['Marker_Color'] = spacex_df['class'].apply(color)
spacex_df.head(5)

,Launch Site,Lat,Long,class,Marker_Color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [22]:
launch_site_df = spacex_df.copy()
launch_site_df.head(10)

,Launch Site,Lat,Long,class,Marker_Color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


In [23]:
spacex_df.head(5)

,Launch Site,Lat,Long,class,Marker_Color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [24]:
for index, record in spacex_df.iterrows():
    print(index, record)

0 Launch Site     CCAFS LC-40
Lat                 28.5623
Long               -80.5774
class                     0
Marker_Color            red
Name: 0, dtype: object
1 Launch Site     CCAFS LC-40
Lat                 28.5623
Long               -80.5774
class                     0
Marker_Color            red
Name: 1, dtype: object
2 Launch Site     CCAFS LC-40
Lat                 28.5623
Long               -80.5774
class                     0
Marker_Color            red
Name: 2, dtype: object
3 Launch Site     CCAFS LC-40
Lat                 28.5623
Long               -80.5774
class                     0
Marker_Color            red
Name: 3, dtype: object
4 Launch Site     CCAFS LC-40
Lat                 28.5623
Long               -80.5774
class                     0
Marker_Color            red
Name: 4, dtype: object
5 Launch Site     CCAFS LC-40
Lat                 28.5623
Long               -80.5774
class                     0
Marker_Color            red
Name: 5, dtype: object
6 Launch S

In [25]:
# Add marker_cluster to current site_map

site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(location = [record['Lat'], record['Long']],
                           popup=record['Launch Site'],
                           icon = folium.Icon(color='white', icon='info-sign', icon_color=record['Marker_Color'])
                          ).add_to(site_map)
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

In [26]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [27]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [28]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [29]:
spacex_df.groupby('Launch Site')[['Lat', 'Long']].first()

,Lat,Long
Launch Site,,
CCAFS LC-40,28.562302,-80.577356
CCAFS SLC-40,28.563197,-80.576820
KSC LC-39A,28.573255,-80.646895
VAFB SLC-4E,34.632834,-120.610746


In [30]:
spacex_df[spacex_df['Launch Site'] == 'CCAFS SLC-40'].iloc[0, 2]

-80.57682003

In [31]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

SLC40_launch_site_lat = spacex_df[spacex_df['Launch Site'] == 'CCAFS SLC-40'].iloc[0, 1]
SLC40_launch_site_lon = spacex_df[spacex_df['Launch Site'] == 'CCAFS SLC-40'].iloc[0, 2]
coastline_lat = 28.56334
coastline_lon = -80.56797

distance_coastline = calculate_distance(SLC40_launch_site_lat, SLC40_launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.8647231887000941

In [32]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline)
        )
    )

site_map.add_child(distance_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [33]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

lines = folium.PolyLine(locations=[[SLC40_launch_site_lat, SLC40_launch_site_lon], 
                                   [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first

**For CCAFS LC-40**  
Markdown the nearest highway, nearest coastline and nearest city

In [34]:
# This time, to get the coordinate of CCAFS LC-40 and the nearest highway, coastline and city
LC40_launch_site_lat = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40'].iloc[0, 1]
LC40_launch_site_lon = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40'].iloc[0, 2]

highway_lat = 28.56246
highway_lon = -80.57065

coastlines_lat = 28.56244
coastlines_lon = -80.56786

railway_lat = 28.57224
railway_lon = -80.58461

# 1) To calculate the distance from launchsite to nearest highway
distance_highway = calculate_distance(LC40_launch_site_lat, LC40_launch_site_lon, highway_lat, highway_lon)
distance_highway

# 2) To calculate the distance from launchsite to nearest coastline
distance_coastline2 = calculate_distance(LC40_launch_site_lat, LC40_launch_site_lon, coastlines_lat, coastlines_lon)
distance_coastline2

# To calculate the distance from launchsite to nearest railway
distance_railway = calculate_distance(LC40_launch_site_lat, LC40_launch_site_lon, railway_lat, railway_lon)
distance_railway

# To add a folium.Marker on your selected closest highway point on the map
distance_marker2 = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway)
        )
    )
distance_marker3 = folium.Marker(
    [coastlines_lat, coastlines_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline2)
        )
    )
distance_marker4 = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway)
        )
    )

# To Create a line between highway coordinates and launch site coordinate
lines2 = folium.PolyLine(locations=[[LC40_launch_site_lat, LC40_launch_site_lon], 
                                   [highway_lat, highway_lon]], weight=1)

lines3 = folium.PolyLine(locations=[[LC40_launch_site_lat, LC40_launch_site_lon], 
                                    [coastlines_lat, coastlines_lon]], weight=1)

lines4 = folium.PolyLine(locations=[[LC40_launch_site_lat, LC40_launch_site_lon], 
                                   [railway_lat, railway_lon]], weight=1)

site_map.add_child(distance_marker2)
site_map.add_child(distance_marker3)
site_map.add_child(distance_marker4)
site_map.add_child(lines2)
site_map.add_child(lines3)
site_map.add_child(lines4)
site_map

A railway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>

A city map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.